In [12]:
import numpy as np
from scipy.special import comb

In [13]:
def permanent(A):
    assert(len(A.shape)==2),"Input should be a 2D matrix"
    assert(A.shape[0]==A.shape[1]),"Input should be a square matrix"
    
    perm=np.float128(0)
    n=A.shape[1]
    C=2**n
    
    for k in range(1,C):
        rowsumprod=np.float128(1)
        chi=np.array(list(np.binary_repr(k))).astype(np.float128)
        for m in range(0,n):
            rowsum=np.float128(0)
            
            for p in range(0,len(chi)):
                rowsum += chi[p]*A[m,len(chi)-p-1]
            
            rowsumprod *= rowsum
            
        perm += ((-1)**(n-chi.sum()))*rowsumprod
    
    return perm

In [14]:
def belief_prop(P,max_iter=50, eps=0.1):
    n=P.shape[0]
    phi=np.sqrt(P)
    m_x2y=np.ones((n,n,n),dtype=np.float128) 
    m_y2x=np.ones((n,n,n),dtype=np.float128) 
    m_x2y_new=np.ones((n,n,n),dtype=np.float128) 
    m_y2x_new=np.ones((n,n,n),dtype=np.float128) 
    belief_xy=np.zeros((n,n,n,n),dtype=np.float128)#belief_xy(i,j,xi,yj) is the value of b(x_i,y_j) for a given value of x_i and y_j
    belief_x=np.zeros((n,n),dtype=np.float128)
    belief_y=np.zeros((n,n),dtype=np.float128)
    
    
    for t in range(max_iter):
        
        #m_x2y_new
        for i in range(n):
            for j in range(n):
                for yj in range(n):
                    sum_xi=0             #sum over all x_i
                    for xi in range(n):
#                         print(i,j,xi,yj,not((xi==j)!=(yj==i)))
                        if not((xi==j)!=(yj==i)): # if psi(x_i,y_j) is non zero
                            prod_m_yk2xi=1     # prod_{k!=j}m_{y_k}(x_i)
                            for k in range(n):
                                if k!=j:
                                    prod_m_yk2xi*=m_y2x[k,i,xi]
                            sum_xi+=phi[i,xi]*prod_m_yk2xi  # the whole summation (over x_i)
                    m_x2y_new[i,j,yj]=sum_xi # message from x_i to y_j for a particular value of y_j
#                     print(i,j,yj,sum_xi)
        m_x2y_new=m_x2y_new/np.sum(m_x2y_new) #normalization
        
        #m_y2x_new
        for j in range(n):
            for i in range(n):
                for xi in range(n):
                    sum_yj=0            
                    for yj in range(n):        
                        if not((xi==j)!=(yj==i)): 
                            prod_m_xk2yj=1     
                            for k in range(n):
                                if k!=i:
                                    prod_m_xk2yj*=m_x2y[k,j,yj]
                            sum_yj+=phi[yj,j]*prod_m_xk2yj  
                    m_y2x_new[j,i,xi]=sum_yj         
        m_y2x_new=m_y2x_new/np.sum(m_y2x_new)#normalization
        
        
        #update using log rule
        for i in range(n):
            for j in range(n):
                for yj in range(n):
                    m_x2y[i,j,yj]=np.exp(np.log(m_x2y[i,j,yj]) + eps*(np.log(m_x2y_new[i,j,yj]) - np.log(m_x2y[i,j,yj])))
#                     print("#",i,j,yj,(m_x2y[i,j,yj]),(m_x2y_new[i,j,yj]))
                    
        for j in range(n):
            for i in range(n):
                for xi in range(n):
                    m_y2x[j,i,xi]=np.exp(np.log(m_y2x[j,i,xi]) + eps*(np.log(m_y2x_new[j,i,xi]) - np.log(m_y2x[j,i,xi])))                   
                    
        m_x2y=m_x2y/np.sum(m_x2y) #normalization
        m_y2x=m_y2x/np.sum(m_y2x)#normalization

        
#         print(t, np.sum(np.abs(m_y2x-m_y2x_new)))

    #compute joint beliefs for each i,j and for each value of x_i and y_j
#     for i in range(n):
#         for j in range(n):
#             for xi in range(n):
#                 for yj in range(n):
#                     if not((xi==j)!=(yj==i)):
#                         prod_m_yk2xi=1     # prod_{k!=j}m_{y_k}(x_i)
#                         prod_m_xl2yj=1
#                         for k in range(n):
#                             if k!=j:
#                                 prod_m_yk2xi*=m_y2x[k,i,xi]
#                         for l in range(n):
#                             if l!=i:
#                                 prod_m_xl2yj*=m_x2y[l,j,yj]
#                         belief_xy[i,j,xi,yj]=phi[i,xi]*phi[yj,j]*prod_m_yk2xi*prod_m_xl2yj

                    
    
#     # normalize the joint beliefs                    
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])
    
    #compute single node beliefs
    for i in range(n):
        for xi in range(n):
            prod_m_yk2xi=1
            for k in range(n):
                prod_m_yk2xi*=m_y2x[k,i,xi]
            belief_x[i,xi]=phi[i,xi]*prod_m_yk2xi
    
    for j in range(n):
        for yj in range(n):
            prod_m_xl2yj=1
            for l in range(n):
                prod_m_xl2yj*=m_x2y[l,j,yj]
            belief_y[j,yj]=phi[yj,j]*prod_m_xl2yj
            
    belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if not((xi==j)!=(yj==i)):
                        belief_xy[i,j,xi,yj]=belief_x[i,xi]*belief_y[j,yj]/(m_x2y[i,j,yj]*m_y2x[j,i,xi])
                    else:
                        belief_xy[i,j,xi,yj]=0
    
    for i in range(n):
        for j in range(n):
            belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])
    
    #normalize single node beliefs
    for i in range(n):
        belief_x[i,:]=belief_x[i,:]/np.sum(belief_x[i,:])
            
    for j in range(n):
        belief_y[j,:]=belief_y[j,:]/np.sum(belief_y[j,:])
        
#     belief_xy=belief_xy/n
#     belief_x=belief_x/(n)
#     belief_y=belief_y/(n)
        
    return m_x2y,m_y2x,belief_xy,belief_x,belief_y,(np.sum(np.abs(m_y2x-m_y2x_new))<1e-10),np.sum(np.abs(m_y2x-m_y2x_new))

In [15]:
def bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x):
        
    t1=t2=t3=t4=t5=t6=0
    n=P.shape[0]
#     belief_x=np.zeros((n,n),dtype=float)
#     belief_y=np.zeros((n,n),dtype=float)
    phi=np.sqrt(P)
    
#     belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=np.outer(belief_x[i,:],belief_y[j,:])/(m_x2y[i,j]*m_y2x[j,i])
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])

    
#     for i in range(n):
#         belief_x[i,:]=np.sum(belief_xy[i,0],axis=1)
#         belief_y[i,:]=np.sum(belief_xy[0,i],axis=0)
        
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if belief_xy[i,j,xi,yj]!=0 and not((xi==j)!=(yj==i)):
                        t1+=belief_xy[i,j,xi,yj]*np.log(phi[i,xi]*phi[yj,j])
                        t2+=belief_xy[i,j,xi,yj]*np.log(belief_xy[i,j,xi,yj])
    
    for i in range(n):
        for xi in range(n):
            if belief_x[i,xi]!=0:
                t3+=belief_x[i,xi]*np.log(belief_x[i,xi])
                t5+=belief_x[i,xi]*np.log(phi[i,xi])
                                          
    for j in range(n):
        for yj in range(n):
            if belief_y[j,yj]!=0:
                t4+=belief_y[j,yj]*np.log(belief_y[j,yj])
                t6+=belief_y[j,yj]*np.log(phi[yj,j])
    
#     print('t1', -t1)    
#     print('t2', t2)
#     print('t3', -(n-1)*t3)
#     print('t4', -(n-1)*t4)
#     print('t5', (n-1)*t5)
#     print('t6', (n-1)*t6)
    energy=-t1+t2-(n-1)*t3-(n-1)*t4+(n-1)*t5+(n-1)*t6
#     print(energy)
    return np.exp(-energy)

In [22]:
def f_bp(P):
    
    def f_bp_p(beta):
        n=P.shape[1]
        z=0
#         print(beta.sum())
        beta=beta.reshape(n,n)
#         print(beta)
        for i in range(n):
            for j in range(n):
                
#                 if beta[i,j]<0:
#                     print(beta[i,j])
                if beta[i,j]!=0 and beta[i,j]!=1 and beta[i,j]>0 and P[i,j]>0: 
                    z+=beta[i,j]*np.log(beta[i,j]/P[i,j])-(1-beta[i,j])*np.log(1-beta[i,j])
#                 print((beta[i,j]/P[i,j]))
        return z
    return f_bp_p

In [29]:
np.random.seed(69)
P=np.random.randint(1000,size=(7,7))+1; #non zero elements
# P=np.ones((5,5))
m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(P,300)

perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
perm=permanent(P)
ub=perm_bethe*np.sqrt(2)**P.shape[0]

print("Perm: \t",perm)
print("Bethe: \t" ,perm_bethe)
print("Bounds: \t",perm_bethe<perm<ub)
print("Convergence: \t",conv)
if not conv:
    print("Conv Value: ", conv_value)
print()
print("Chertkov")
f=f_bp(P)
perm_bethe_wc=np.exp(-f(belief_x))
ub_wc=perm_bethe_wc*np.sqrt(2)**P.shape[0]
print("Bethe WC: \t" ,perm_bethe_wc)
print("Bounds: \t",perm_bethe_wc<perm<ub_wc)
print("Difference: \t", np.abs(perm_bethe-perm_bethe_wc))
print("Difference %: \t", np.abs(perm_bethe-perm_bethe_wc)/((perm_bethe+perm_bethe_wc)/2))

Perm: 	 2.6790026134057470919e+22
Bethe: 	 7.028723468426812217e+21
Bounds: 	 True
Convergence: 	 True

Chertkov
Bethe WC: 	 7.028723468700632373e+21
Bounds: 	 True
Difference: 	 3.8957309622532854495e-11


In [21]:
#relation between chertkov and hj

-244.46264293603599072

In [11]:
belief_y

array([[0.0179236 , 0.28138636, 0.25659671, 0.08526026, 0.35883307],
       [0.21550912, 0.34044494, 0.28208485, 0.03039895, 0.13156214],
       [0.36442671, 0.10496507, 0.10703008, 0.01006182, 0.41351632],
       [0.24586882, 0.00685304, 0.13033512, 0.5960525 , 0.02089052],
       [0.15627175, 0.26635059, 0.22395323, 0.27822649, 0.07519794]],
      dtype=float128)

In [29]:
np.random.seed(9)
P=np.random.randint(100,size=(5,10))+1; #non zero elements
# P=np.ones((5,5))
n=P.shape[0]
m=P.shape[1]
Q=np.ones((m-n,m))
P=np.append(P,Q,axis=0);
P

array([[ 93.,  55.,  57.,  23.,  66.,  23.,  53.,  60.,  41.,  92.],
       [ 34.,  94.,  93.,   1.,  61.,  60.,  89.,  75.,  57.,  94.],
       [ 63.,  13.,  19.,  87.,  57.,   2.,  57., 100.,  12.,  38.],
       [ 20.,  10.,  81.,  57.,  50.,  64.,  14.,  60.,  59.,  34.],
       [ 78.,  94.,  55.,  64.,  23.,  51.,  41.,   5., 100.,  53.],
       [  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.],
       [  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.]])

In [ ]:
m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(P,100)

In [34]:
perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)/120
perm=permanent(P)/120
ub=perm_bethe*np.sqrt(2)**5

print("Perm: \t",perm)
print("Bethe: \t" ,perm_bethe)
print("ub: \t", ub)
print("Bounds: \t",perm_bethe<perm<ub)
print("Convergence: \t",conv)
if not conv:
    print("Conv Value: ", conv_value)

Perm: 	 13394705527549.0
Bethe: 	 2836502600587.843429
ub: 	 16045681789831.535839
Bounds: 	 True
Convergence: 	 False
Conv Value:  0.0005326398239666902556
